In [5]:
pip install -U sentence-transformers

In [6]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer, models
from sklearn.model_selection import train_test_split
from sentence_transformers import InputExample

Reading the data from the csv file on Drive. First, connect to drive. Second, read the data.

In [7]:
!pip install google-colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df = pd.read_csv('/content/drive/MyDrive/BİL495 496/Data/DataForFineTuning.csv')
df.head()

,S1,S2,Similarity
0,An algorithm is a set of instructions to solve...,An algorithm provides the logic and methodolog...,0.9396
1,Big O notation describes the upper bound or wo...,Big O notation quantifies the upper bound on t...,0.9109
2,Cycle detection in a directed graph can be don...,Detecting a cycle in a directed graph is a com...,0.8626
3,Breadth-first search (BFS) can be implemented ...,Implementing a Breadth-First Search (BFS) algo...,0.7243
4,A priority queue can be implemented using a he...,Implementing a priority queue using a heap inv...,0.8761


Prepare dataset. First, split datas of the types computer science and general questions. Then, choose 30% of each of them as test and remaining train data. Lastly, organize data types properly for training.

In [9]:
df_cs = df.iloc[:942,:]
df_general = df.iloc[942:,:]

train_cs, test_cs = train_test_split(df_cs, test_size=0.3)
train_general, test_general = train_test_split(df_general, test_size=0.3)

train_df = pd.concat([train_cs, train_general], axis=0)
test_df = pd.concat([test_cs, test_general], axis=0)

In [10]:
train_df_dict = train_df.to_dict('list')

In [11]:
test_df_dict = test_df.to_dict('list')
#print(test_df_dict[0])

In [12]:
pip install datasets

In [13]:
import datasets
from datasets import Dataset

dataset_train = Dataset.from_dict(train_df_dict)
dataset_test = Dataset.from_dict(test_df_dict)
dd = datasets.DatasetDict({"train":dataset_train,"test":dataset_test})

In [14]:
print(f"- The dataset has {dd['train'].num_rows} examples.")
print(f"- Each example is a {type(dd['train'][0])} with a {type(dd['train'][0])} as value.")
print(f"- Examples look like this: {dd['train'][0]}")

- The dataset has 726 examples.
- Each example is a <class 'dict'> with a <class 'dict'> as value.
- Examples look like this: {'S1': 'Synchronous programming executes tasks sequentially, while asynchronous programming allows tasks to run independently, improving performance and responsiveness.', 'S2': 'Synchronous programming executes tasks sequentially, while asynchronous programming allows non-blocking execution, enabling tasks to run concurrently.', 'Similarity': 0.9738}


Create InputExamples for training.

In [15]:
from sentence_transformers import InputExample

train_examples = []
train_data = dd['train']
# For agility we only 1/2 of our available data
n_examples =  dd['train'].num_rows // 2

for i in range(n_examples):
  example = train_data[i]
  train_examples.append(InputExample(texts=[example['S1'], example['S2']], label=example['Similarity']))

In [16]:
print(f"We have a {type(train_examples)} of length {len(train_examples)} containing {type(train_examples[0])}'s.")

We have a <class 'list'> of length 363 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.


Create DataLoader for training.

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

Get the base pre-trained model for training.

In [18]:
model = SentenceTransformer("neuml/pubmedbert-base-embeddings")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Define loss function according to label (which is float in our data).

In [19]:
from sentence_transformers import losses

train_loss = losses.CosineSimilarityLoss(model=model)

Arrange parameters.

In [20]:
num_epochs = 10

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

Train the model.

In [21]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Save model after the training.

In [23]:
from huggingface_hub import notebook_login

notebook_login()

In [24]:
model.save_to_hub("pubmedbert-base-embeddings-base-sentence-transformer")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/eladogruyol/pubmedbert-base-embeddings-base-sentence-transformer/commit/cc218254b1a94ea4350b2cdaec0430b48e563ba0'

Load the trained model with sentence-transformers. Try an example.

In [27]:
from sentence_transformers import SentenceTransformer, util
sentence1 = ["This is an example sentence"]
sentence2 = ["Each sentence is converted"]

#model = SentenceTransformer("neuml/pubmedbert-base-embeddings")
#model = SentenceTransformer("all-mpnet-base-v2")
saved_model = SentenceTransformer("eladogruyol/pubmedbert-base-embeddings-base-sentence-transformer")
embeddings1 = saved_model.encode(sentence1)
embeddings2 = saved_model.encode(sentence2)

Results of the example:

In [26]:
cosine_scores = util.cos_sim(embeddings1, embeddings2)
print(cosine_scores[0][0])

tensor(0.6939)


Test data overview:

In [30]:
test_df.head()

,S1,S2,Similarity
153,Microservices architecture breaks down a large...,"Advantages include scalability, flexibility, a...",0.5618
568,PCA reduces the dimensionality of data by tran...,PCA works by transforming high-dimensional dat...,0.8886
913,Autoboxing is the automatic conversion of prim...,Autoboxing simplifies the process of converti...,0.9392
170,A circular linked list is a variation of a li...,A circular linked list is a linked list where ...,0.9442
454,"The ""try-with-resources"" statement in Java is ...","The ""try-with-resources"" statement automatical...",0.7489


Give test sentences to model. Find sentence similarities.

In [32]:
a1_list = test_df['S1'].values.tolist()
a2_list = test_df['S2'].values.tolist()

embeddings1 = saved_model.encode(a1_list)
embeddings2 = saved_model.encode(a2_list)

In [33]:
cosine_scores = util.cos_sim(embeddings1, embeddings2)
results = []

for i in range(len(a1_list)):
  #print(cosine_scores[i][i])
  result = []
  result.append(a1_list[i])
  result.append(a2_list[i])
  result.append(cosine_scores[i][i])
  results.append(result)

print(results[:10])

[['Microservices architecture breaks down a large application into smaller, independent services. Advantages include scalability and easier maintenance, while disadvantages may include increased complexity and potential communication overhead.', 'Advantages include scalability, flexibility, and easier maintenance. Disadvantages may include increased complexity, higher resource consumption, and challenges in managing distributed systems.', tensor(0.7232)], ['PCA reduces the dimensionality of data by transforming it into a new coordinate system, capturing the most significant variations in the data.', 'PCA works by transforming high-dimensional data into a lower-dimensional space while retaining the most important information, achieved through eigenvalue decomposition of the covariance matrix to find principal components that capture the maximum variance in the data.', tensor(0.9235)], ['Autoboxing is the automatic conversion of primitive data types into their corresponding wrapper class

Write results in a different csv file (here, "results.csv").

In [34]:
import csv
with open('/content/drive/MyDrive/BİL495 496/Data/trained_model_results.csv', 'w', newline='') as file:
    # Step 4: Using csv.writer to write the list to the CSV file
    writer = csv.writer(file)
    writer.writerows(results)